In [ ]:
# How to read data from google sheets
# https://www.youtube.com/watch?v=hyUw-koO2DA

In [5]:
# !pip install gspread
# !pip install oauth2client
!pip install PyOpenSSL

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/home/ubuntu/finances/raw_data/teste-274223-f75546e9a58b.json',
    scopes=scopes
)
files = gspread.authorize(creds)
workbook = files.open('Planilha de Controle de Carteira CDV v.6.0e')
worksheet = workbook.worksheet('2 - Renda Variável')

# Get a pandas dataframe from the worksheet object
df = pd.DataFrame(worksheet.get_all_records(
    expected_headers=[
        "",
        "Data",
        "Mês",
        "Papel",
        "Qtde",
        "Preço",
        "Taxas",
        "IRRF",
        "Montante sem corretagem",
        "Montante com corretagem",
        "Preço com corretagem",
        "Estoque posição",
        "Estoque preço médio",
        "Compra ou Venda?",
        "Corretora",
    ],
))

In [79]:
from datetime import datetime

df2 = df.query("Data != ''")[["Data", "Papel", "Qtde", "Preço", "Taxas"]].rename(
    columns={
        "Data": "date",
        "Papel": "ticker",
        "Qtde": "quantity",
        "Preço": "price",
        "Taxas": "taxes"
    }
).assign(_processed_at=datetime.now())

# .astype(
#     {
#     "ticker": str,
#     "quantity": int,
#     "price": "float64",
#     "taxes": "float64"}
# )

In [49]:
df2["quantity"] = df2["quantity"].apply(lambda x: x.replace("(", "-").replace(")", "").replace(".", "") if type(x) == str else x).astype(int)

In [ ]:
df2["price"] = df2["price"].str.replace("R$ ", "").str.replace(",", ".").astype(float)


In [67]:
df2["taxes"] = df2["taxes"].str.replace("R$ ", "").str.replace(",", ".").str.replace("-", "0").astype(float)

In [70]:
df2[df2["quantity"] < 0]

,date,ticker,quantity,price,taxes,_processed_at
62,01/08/2023,CBAV3,-100,5.33,0.16,2025-04-19 21:13:05.461248
63,01/08/2023,CBAV3,-35,5.32,0.06,2025-04-19 21:13:05.461248
64,01/08/2023,CCRO3,-38,13.27,0.15,2025-04-19 21:13:05.461248
65,01/08/2023,CCRO3,-11,13.26,0.04,2025-04-19 21:13:05.461248
66,01/08/2023,CSUD3,-40,15.36,0.18,2025-04-19 21:13:05.461248
67,01/08/2023,CSUD3,-13,15.35,0.06,2025-04-19 21:13:05.461248
68,01/08/2023,JBSS3,-39,19.15,0.22,2025-04-19 21:13:05.461248
69,01/08/2023,LEVE3,-15,45.00,0.20,2025-04-19 21:13:05.461248
70,01/08/2023,TECN3,-100,3.47,0.10,2025-04-19 21:13:05.461248
71,01/08/2023,TECN3,-95,3.46,0.10,2025-04-19 21:13:05.461248


In [1]:
import pandas as pd
dfinal = pd.read_csv("/home/ubuntu/finances/raw_data/Planilha de Controle de Carteira CDV v.6.0e - Página22.csv", header=None)

dfinal.columns = ["date", "ticker", "quantity", "price", "taxes"]

# How to read a csv without columns names?
dfinal["quantity"] = dfinal["quantity"].str.replace(".", "").str.replace(",", ".").astype('float64')
dfinal["price"] = dfinal["price"].str.replace(".", "").str.replace(",", ".").astype('float64')
dfinal["taxes"] = dfinal["taxes"].str.replace(".", "").str.replace(",", ".").astype('float64')

In [4]:
from utils import persist_dataframe_to_database

In [15]:
dfinal["date"] = pd.to_datetime(dfinal["date"], format="%d/%m/%Y")


In [16]:
persist_dataframe_to_database(dfinal, "stocks", "transactions", assign_processed_at_column=True)